In [ ]:
from boututils.datafile import DataFile
from boutdata.collect import collect
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, sys, pathlib
import platform
import traceback
import xarray as xr
import xbout
from pathlib import Path
import xhermes as xh

sys.path.append(os.path.join(r"/users/jlb647/scratch/simulation_program/hermes-3_sim/sdtool_load_test/sdtools"))
sys.path.append(os.path.join(r"/users/jlb647/scratch/simulation_program/hermes-3_sim/analysis/my_notebooks/notebooks/hermes-3/transients"))
sys.path.append(os.path.join(r"/users/jlb647/scratch/simulation_program/hermes-3_sim/analysis/my_notebooks/notebooks/hermes-3/general_functions"))


from plotting_functions import *
from convergence_functions import * 

from hermes3.case_db import *
from hermes3.casedeck import*
from hermes3.load import *
from hermes3.named_selections import *
from hermes3.plotting import *
from hermes3.grid_fields import *
from hermes3.accessors import *
from hermes3.utils import *
from hermes3.fluxes import *
from hermes3.selectors import *
from hermes3.balance1d import *

# plt.style.use('ggplot')
plt.rcParams.update({'font.size': 10})
linewidth = 3
markersize = 15



# plt.style.use('ggplot')
plt.style.use('default')
plt.rcParams["axes.edgecolor"] = "black"
plt.rcParams["axes.linewidth"] = 1
plt.rcParams['xtick.labelsize'] = 18
plt.rcParams['ytick.labelsize'] = 18
plt.rcParams['axes.grid'] = True
plt.rcParams.update({'font.size': 16})



%load_ext autoreload
%autoreload 2


: 

# 2D

In [ ]:
db = CaseDB(case_dir = '/users/jlb647/scratch/simulation_program/hermes-3_sim/simulation_dir/2025-01_STEP_1D-2D_comparison/Mike_2D_cases/m4ab-tune_albedo_new_branch', grid_dir='/users/jlb647/scratch/simulation_program/hermes-3_sim/sdtool_load_test/grids')

case = db.load_case_2D("m4ab-tune_albedo_new_branch", use_squash = True, verbose = True)

# 1D

In [ ]:
fl_ex_1D = xh.open('/users/jlb647/scratch/simulation_program/hermes-3_sim/simulation_dir/2025-01_STEP_1D-2D_comparison/Include_flux_expansion/m4ab-tune_albedo_1D_sep_add_1_working_add_flux_expansion')

# 1D flux profile

In [ ]:

offset = -150
fig, ax = plt.subplots(1, 1, figsize = (10, 6))
plt.plot(fl_ex_1D['y'][:offset].values, fl_ex_1D['J'][0][:offset].values, label = '1D', linewidth = linewidth)

# 2D flux profile

In [ ]:
params = ['R', 'Bxy']
profile = get_1d_poloidal_data(case.ds.isel(t=-1), params = params, region = ('outer_lower'), sepadd = 1)

xpoint_location = profile['R'].idxmin()
B_xpoint = profile['Bxy'][xpoint_location]

f_expansion = ( -1* profile['Bxy']/B_xpoint ) + 2

x_sparce = profile['Spar']


# fig,ax = plt.subplots(1,1, figsize = (10,5))
# ax.plot(profile['Spar'], f_expansion, label = 'f_expansion', marker = 'o')

# print(x_sparce)

In [ ]:
x_old = profile['Spar'].values

x_rebase = (x_old/x_old.max())*2*np.pi

fig,ax = plt.subplots(1,1, figsize = (10,5))
ax.plot(x_rebase, f_expansion, label = 'f_expansion', marker = 'o')

## Fitting procedure for 2D data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Replace these with your actual data


x_data = x_rebase  # Add your x values here
y_data = f_expansion

# Ensure you have data before proceeding
if len(x_data) == 0 or len(y_data) == 0:
    raise ValueError("Please provide your x_data and y_data arrays.")

# Fit a 10th-degree polynomial
coefficients = np.polyfit(x_data, y_data, 10)

# Generate the polynomial function
polynomial = np.poly1d(coefficients)

# Evaluate the polynomial for plotting
x_fit = np.linspace(min(x_data), max(x_data), 500)
y_fit = polynomial(x_fit)

# Print coefficients
print("Fitted Polynomial Coefficients:")
for i, coef in enumerate(coefficients[::-1]):
    print(f"coef{i}= {coef:.6f}")

# Plot the data and the fitted polynomial
plt.figure(figsize=(10, 6))
plt.plot(x_data, y_data, 'o', label="B expansion", markersize=5)
plt.plot(x_fit, y_fit+0.01, '-', label="10th degree Polynomial Fit")
plt.ylabel(r'$B/B_{xpoint}$')
plt.xlabel(r"$S_{\parallel} (0 , 2\pi)$")
plt.legend()
plt.grid(True)
plt.show()


# Rebase to 1D

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

# Load the original grid and function values
# (Assume you have original `x_old` and function `f_old` fitted to it)
x_data = x_rebase  # Add your x values here
y_data = f_expansion     # Example function fitted on `x_old`

# Define the new grid (non-uniform)
ny = 400
length = 2 * np.pi
dymin = 0.1  # Minimum spacing factor

x_new = fl_ex_1D['y'].values
x_new[0] = 0

for i in range(ny):
    y = x_new[i]
    dy = (length / ny) * (1 + (1 - dymin) * (1 - y / np.pi))
    x_new[i + 1] = x_new[i] + dy

x_new[-1] = 2 * np.pi  # Ensure it ends at 2π

# Interpolate the function onto the new grid
interp_func = interp1d(x_old, f_old, kind="cubic", fill_value="extrapolate")

# Apply function to the new grid
f_new = interp_func(x_new)

# Plot comparison
plt.figure(figsize=(8, 5))
plt.scatter(x_old, f_old, label="Original Function (Fitted Grid)", s=10)
plt.plot(x_new, f_new, label="Function Applied to New Grid", color="orange")
plt.xlabel("$S_{||} (0, 2\pi)$")
plt.ylabel("$B / B_{xpoint}$")
plt.legend()
plt.grid()
plt.show()


# Matt flux expansion

In [ ]:
fe_matt = xh.open('/users/jlb647/scratch/simulation_program/hermes-3_sim/simulation_dir/2025-01_STEP_1D-2D_comparison/Include_flux_expansion/m4ab-tune_albedo_1D_sep_add_1_working_add_flux_expansion_matt')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (10, 6))
plt.plot(fe_matt['y'], fe_matt['J'][0], label = '1D', linewidth = linewidth, marker = 'o')

In [ ]:
# Matts version for STEP
coef0 = 1.7597676684e+00		# Manually setting the flux expansion with a profile from the equilibrium files
coef1 = 4.1883560155e-01		# Very hacky but it gets the job done
coef2 = -4.6906367245e+00		# area_expansion normally does a good enough job
coef3 = 7.3174700447e+00
coef4 = -5.7251396733e+00
coef5 = 2.6799740048e+00
coef6 = -7.9457620000e-01
coef7 = 1.5075033673e-01
coef8 = -1.7757965412e-02
coef9 = 1.1834284917e-03
coef10 = -3.4102000406e-05

y = 


J = 0  + y**0*coef0 + y**1*coef1 + y**2*coef2 + y**3*coef3 + y**4*coef4 + y**5*coef5 + y**6*coef6 + y**7*coef7 + y**8*coef8 + y**9*coef9 + y**10*coef10